In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import load_model
import warnings
import nltk
from nltk.corpus import stopwords
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('gujarati-train.csv')
df.head()

,headline,Label
0,IPL Auction 2022: 12-13 ફેબ્રુઆરીએ કેટલા વાગ્ય...,Sports
1,રોહિત-કોહલીના વિવાદ પર ગાવસ્કરે કર્યો મોટો ખુલ...,Sports
2,"પાકિસ્તાની ફાસ્ટ બોલરને કરાયો સસ્પેંડ, 155kphન...",Sports
3,એશિઝમાં કારમી હારને લીધે ગિલ્સ બાદ મુખ્ય કોચને...,Sports
4,ક્યાં રમાશે IPL 2022ના મેચ? સૌરવ ગાંગુલીએ આપ્ય...,Sports


In [3]:
def count_length():
    df['word_count'] = df['headline'].apply(lambda x: len(str(x).split(" ")))

In [4]:
count_length()

In [5]:
df.head(10)

,headline,Label,word_count
0,IPL Auction 2022: 12-13 ફેબ્રુઆરીએ કેટલા વાગ્ય...,Sports,10
1,રોહિત-કોહલીના વિવાદ પર ગાવસ્કરે કર્યો મોટો ખુલ...,Sports,10
2,"પાકિસ્તાની ફાસ્ટ બોલરને કરાયો સસ્પેંડ, 155kphન...",Sports,10
3,એશિઝમાં કારમી હારને લીધે ગિલ્સ બાદ મુખ્ય કોચને...,Sports,11
4,ક્યાં રમાશે IPL 2022ના મેચ? સૌરવ ગાંગુલીએ આપ્ય...,Sports,10
5,"ટીમ ઈન્ડિયાનો આ ખેલાડી વેડફી રહ્યો છે ટેલેન્ટ,...",Sports,11
6,"IND vs WI: ભારતીય ટીમમાં કોરોનાનો વિસ્ફોટ, ફરી...",Sports,15
7,IPL 2022: હરાજીમાં આ ખેલાડીઓનું વેચાવવું છે મુ...,Sports,12
8,IPL 2022ના મેગા ઓક્શનમાં અનસોલ્ડ રહી શકે છે આ ...,Sports,17
9,IPL 2022: ક્યા દેશના કેટલા ખેલાડી લઇ રહ્યાં છે...,Sports,12


In [6]:
f = open("Gujarati_stopwords.txt", "r",encoding="utf8")
stopwords_gu = []
for i in f:
    i=i.split("\n")
    stopwords_gu.append(str(i[0]))
stopwords_en = stopwords.words('english')
punctuations = [".",",","?","/","<",">","!",":",";","'","[","]","{","}","(",")","-","@","#","$","%","^","&","*"]
stopwords_lst = stopwords_gu + stopwords_en + punctuations

In [7]:
df['label_id'] = df.Label.factorize()[0]

def preprocess_text(text):
    text = text.lower()
    text = re.sub('[a-zA-Z0-9]', '',text)
    text = re.sub("((www.[^s]+)|(https?://[^s]+))",'',text)
    text = re.sub("@[^s]+",'',text)
    text = re.sub("[s]+", ' ', text)
    text = re.sub(r"#([^s]+)", r'', text)
    text = text.strip('"')
    return text 

In [8]:
df['headline'] = df.headline.apply(lambda x: preprocess_text(x))
count_length()

In [9]:
df.head(10)

,headline,Label,word_count,label_id
0,: - ફેબ્રુઆરીએ કેટલા વાગ્યાથી લાગશે ખેલાડીઓન...,Sports,10,0
1,રોહિત-કોહલીના વિવાદ પર ગાવસ્કરે કર્યો મોટો ખુલ...,Sports,10,0
2,"પાકિસ્તાની ફાસ્ટ બોલરને કરાયો સસ્પેંડ, ની સ્પિ...",Sports,10,0
3,એશિઝમાં કારમી હારને લીધે ગિલ્સ બાદ મુખ્ય કોચને...,Sports,11,0
4,ક્યાં રમાશે ના મેચ? સૌરવ ગાંગુલીએ આપ્યો આ જવાબ,Sports,10,0
5,"ટીમ ઈન્ડિયાનો આ ખેલાડી વેડફી રહ્યો છે ટેલેન્ટ,...",Sports,11,0
6,": ભારતીય ટીમમાં કોરોનાનો વિસ્ફોટ, ફરી પ્રથમ ...",Sports,15,0
7,": હરાજીમાં આ ખેલાડીઓનું વેચાવવું છે મુશ્કેલ, ...",Sports,12,0
8,ના મેગા ઓક્શનમાં અનસોલ્ડ રહી શકે છે આ વિદેશી...,Sports,17,0
9,: ક્યા દેશના કેટલા ખેલાડી લઇ રહ્યાં છે મેગા ઓ...,Sports,12,0


In [10]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens

In [11]:
df['headline'] = df.headline.apply(lambda x: tokenization(x))
df.head(10)

,headline,Label,word_count,label_id
0,"[:, -, ફેબ્રુઆરીએ, કેટલા, વાગ્યાથી, લાગશે, ખેલ...",Sports,10,0
1,"[રોહિત, -, કોહલીના, વિવાદ, પર, ગાવસ્કરે, કર્યો...",Sports,10,0
2,"[પાકિસ્તાની, ફાસ્ટ, બોલરને, કરાયો, સસ્પેંડ, ,,...",Sports,10,0
3,"[એશિઝમાં, કારમી, હારને, લીધે, ગિલ્સ, બાદ, મુખ્...",Sports,11,0
4,"[ક્યાં, રમાશે, ના, મેચ, ?, સૌરવ, ગાંગુલીએ, આપ્...",Sports,10,0
5,"[ટીમ, ઈન્ડિયાનો, આ, ખેલાડી, વેડફી, રહ્યો, છે, ...",Sports,11,0
6,"[:, ભારતીય, ટીમમાં, કોરોનાનો, વિસ્ફોટ, ,, ફરી,...",Sports,15,0
7,"[:, હરાજીમાં, આ, ખેલાડીઓનું, વેચાવવું, છે, મુશ...",Sports,12,0
8,"[ના, મેગા, ઓક્શનમાં, અનસોલ્ડ, રહી, શકે, છે, આ,...",Sports,17,0
9,"[:, ક્યા, દેશના, કેટલા, ખેલાડી, લઇ, રહ્યાં, છે...",Sports,12,0


In [12]:
for i in range(len(df)):
    df['headline'][i]=[ele for ele in df['headline'][i] if ele not in (stopwords_lst)]
count_length()
df.head(10)

,headline,Label,word_count,label_id
0,"[ફેબ્રુઆરીએ, કેટલા, વાગ્યાથી, લાગશે, ખેલાડીઓની...",Sports,6,0
1,"[રોહિત, કોહલીના, વિવાદ, ગાવસ્કરે, કર્યો, મોટો,...",Sports,10,0
2,"[પાકિસ્તાની, ફાસ્ટ, બોલરને, કરાયો, સસ્પેંડ, સ્...",Sports,7,0
3,"[એશિઝમાં, કારમી, હારને, લીધે, ગિલ્સ, બાદ, મુખ્...",Sports,10,0
4,"[ક્યાં, રમાશે, મેચ, સૌરવ, ગાંગુલીએ, જવાબ]",Sports,6,0
5,"[ટીમ, ઈન્ડિયાનો, ખેલાડી, વેડફી, રહ્યો, ટેલેન્ટ...",Sports,9,0
6,"[ભારતીય, ટીમમાં, કોરોનાનો, વિસ્ફોટ, મેચ, નહી, ...",Sports,10,0
7,"[હરાજીમાં, ખેલાડીઓનું, વેચાવવું, મુશ્કેલ, કરોડ...",Sports,7,0
8,"[મેગા, ઓક્શનમાં, અનસોલ્ડ, રહી, શકે, વિદેશી, ખે...",Sports,10,0
9,"[ક્યા, દેશના, કેટલા, ખેલાડી, લઇ, રહ્યાં, મેગા,...",Sports,8,0


In [13]:
df['headline'] = df['headline'].str.join(" ")
df.head(10)

,headline,Label,word_count,label_id
0,ફેબ્રુઆરીએ કેટલા વાગ્યાથી લાગશે ખેલાડીઓની બોલી,Sports,6,0
1,રોહિત કોહલીના વિવાદ ગાવસ્કરે કર્યો મોટો ખુલાસો...,Sports,10,0
2,પાકિસ્તાની ફાસ્ટ બોલરને કરાયો સસ્પેંડ સ્પિડથી ...,Sports,7,0
3,એશિઝમાં કારમી હારને લીધે ગિલ્સ બાદ મુખ્ય કોચને...,Sports,10,0
4,ક્યાં રમાશે મેચ સૌરવ ગાંગુલીએ જવાબ,Sports,6,0
5,ટીમ ઈન્ડિયાનો ખેલાડી વેડફી રહ્યો ટેલેન્ટ ગાવસ્...,Sports,9,0
6,ભારતીય ટીમમાં કોરોનાનો વિસ્ફોટ મેચ નહી રમી શકે...,Sports,10,0
7,હરાજીમાં ખેલાડીઓનું વેચાવવું મુશ્કેલ કરોડોમાં ...,Sports,7,0
8,મેગા ઓક્શનમાં અનસોલ્ડ રહી શકે વિદેશી ખેલાડી કર...,Sports,10,0
9,ક્યા દેશના કેટલા ખેલાડી લઇ રહ્યાં મેગા ઓક્શનમાં,Sports,8,0


In [14]:
X = df['headline'].values
Y = df['label_id']
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [15]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20)
max_words = 10000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(7,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model
model = RNN()

In [17]:
test_classes = []
for i in range(len(Y_test)):
    for j in range(len(Y_test[i])):
        test_classes.append(Y_test[i][j]) 

In [18]:
model.compile(loss='sparse_categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
history = model.fit(sequences_matrix,Y_train,batch_size=256,epochs=10,
          validation_split=0.2)

Epoch 1/10
95/95 [==============================] - 54s 536ms/step - loss: 1.3412 - accuracy: 0.4202 - val_loss: 0.7652 - val_accuracy: 0.7482
Epoch 2/10
95/95 [==============================] - 49s 519ms/step - loss: 0.5340 - accuracy: 0.8100 - val_loss: 0.4264 - val_accuracy: 0.8495
Epoch 3/10
95/95 [==============================] - 53s 552ms/step - loss: 0.3328 - accuracy: 0.8869 - val_loss: 0.3694 - val_accuracy: 0.8720
Epoch 4/10
95/95 [==============================] - 57s 596ms/step - loss: 0.2139 - accuracy: 0.9296 - val_loss: 0.3403 - val_accuracy: 0.8961
Epoch 5/10
95/95 [==============================] - 51s 538ms/step - loss: 0.1699 - accuracy: 0.9438 - val_loss: 0.3564 - val_accuracy: 0.8905
Epoch 6/10
95/95 [==============================] - 58s 611ms/step - loss: 0.1350 - accuracy: 0.9566 - val_loss: 0.3202 - val_accuracy: 0.9034
Epoch 7/10
95/95 [==============================] - 62s 654ms/step - loss: 0.1124 - accuracy: 0.9649 - val_loss: 0.3317 - val_accuracy: 0.9045

In [19]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
accr = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.2f}'.format(accr[1]))

237/237 [==============================] - 9s 40ms/step - loss: 0.4011 - accuracy: 0.8962
Accuracy: 0.90


In [20]:
model.save('GU_LSTM.h5')

In [21]:
predicted_categories = []
preds = model.predict(test_sequences_matrix)
for i in preds:
    pred_classes = np.argsort(i)[-1:][::-1]
    predicted_categories.append(pred_classes[0])

In [22]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
report = classification_report(test_classes, predicted_categories)
print(report)

              precision    recall  f1-score   support

           0       0.91      0.91      0.91      1562
           1       0.87      0.83      0.85      1423
           2       0.84      0.87      0.85      1502
           3       0.92      0.94      0.93      1588
           4       0.93      0.92      0.93      1481

    accuracy                           0.90      7556
   macro avg       0.90      0.89      0.90      7556
weighted avg       0.90      0.90      0.90      7556



In [23]:
mat = confusion_matrix(test_classes, predicted_categories)
mat

array([[1426,   28,   24,   58,   26],
       [  29, 1183,  171,   16,   24],
       [  29,  110, 1303,   26,   34],
       [  45,    7,   24, 1500,   12],
       [  32,   28,   33,   28, 1360]], dtype=int64)

In [24]:
id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

In [25]:
sample_text1 = ["શેરમાર્કેટમાં કડાકો:10 દિવસમાં Mcap 15 લાખ કરોડ ઘટ્યું; સેન્સેક્સ 678 પોઇન્ટ તૂટ્યો, ઇન્વેસ્ટર્સે 3 દિવસમાં 6.15 લાખ ગુમાવ્યાં"]

txts = tok.texts_to_sequences(sample_text1)
txts = sequence.pad_sequences(txts, maxlen=max_len)
preds = model.predict(txts)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

business 100.00 %


In [26]:
sample_text2 = ["SRKના દીકરાની ઘરવાપસી LIVE:આર્યન ખાનને 'મન્નત'થી આર્થર રોડ જેલનું 13 કિલોમીટરનું અંતર કાપતાં 28 દિવસ થયા"]

txts = tok.texts_to_sequences(sample_text2)
txts = sequence.pad_sequences(txts, maxlen=max_len)
preds = model.predict(txts)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

entertainment 100.00 %


In [27]:
sample_text3 = ["લોન્ચ થયો Oppo નો Waterproof સ્માર્ટફોન, ધાંસૂ કેમેરા સાથે મળશે ગજબના ફીચર્સ"]

txts = tok.texts_to_sequences(sample_text3)
txts = sequence.pad_sequences(txts, maxlen=max_len)
preds = model.predict(txts)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

tech 100.00 %


In [28]:
sample_text4 = ["ઉત્તરાખંડમાં ભાજપને મોટો ઝટકો, કેબિનેટ મંત્રી પદેથી રાજીનામું આપીને આ નેતા કોંગ્રેસમાં થયા સામેલસ"]

txts = tok.texts_to_sequences(sample_text4)
txts = sequence.pad_sequences(txts, maxlen=max_len)
preds = model.predict(txts)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

politics 100.00 %


In [29]:
sample_text5 = ["સાઉથ આફ્રિકન સ્પિનર શેન વ્હાઈટહેડે ઈનિંગમાં ૧૦ વિકેટ ઝડપી ઈતિહાસ રચ્યો"]

txts = tok.texts_to_sequences(sample_text5)
txts = sequence.pad_sequences(txts, maxlen=max_len)
preds = model.predict(txts)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

sports 100.00 %
